In [1]:
import sqlite3
from sqlite_utils import Database
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual 
from ipywidgets import VBox, HBox, Button, RadioButtons
import ipywidgets as widgets
from math import floor,ceil
%matplotlib widget


#INTER_DB = Database("dlma-vac.stl.db")
#SCAD_DB = Database("dlma_openscad_20211209.stl_intersections.db")
plot1 = {}
plot2 = {}
plot1['dlma'] = "dlma-vac.stl_intersections.db"
plot1['dlmb'] = "DLMB-vac.stl_intersections.db"
plot1['qmqa'] = "QMQA-vac.stl_intersections.db"
plot1['qmqb'] = "QMQB-vac.stl_intersections.db"
plot1['fodo'] = "FODO-vac.stl_intersections.db"
#plot2['dlma'] = "dlma_20211220.stl_intersections.db"
plot2['dlma'] = "dlma_20220131.stl_intersections.db"
plot2['dlmb'] = "dlmb_20220110.stl_intersections.db"
plot2['qmqa'] = "qmqa-20220114.stl_intersections.db"
plot2['qmqb'] = "cube.stl_intersections.db"
plot2['fodo'] = "fodo_20220123.stl_intersections.db"

vacuum_module = 'fodo'
PLOT_1_DB = Database(plot1[vacuum_module])
PLOT_2_DB = Database(plot2[vacuum_module])
SLICE_AXIS = "Z"


# Determine the variables
x_plot_parameters = {}
y_plot_parameters = {}
z_plot_parameters = {}

if SLICE_AXIS == "X":
    x_min_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'y_min';"
    x_max_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'y_max';"
    y_min_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'z_min';"
    y_max_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'z_max';"
    
if SLICE_AXIS == "Y":
    x_min_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'z_min';"
    x_max_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'z_max';"
    y_min_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'x_min';"
    y_max_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'x_max';"
    
if SLICE_AXIS == "Z":
    x_min_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'x_min';"
    x_max_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'x_max';"
    y_min_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'y_min';"
    y_max_query = "SELECT value FROM " + SLICE_AXIS + "_variables where name = 'y_max';"


rows = PLOT_1_DB.query(x_min_query)
for row in rows:
    x_plot_parameters['center_min'] = row['value']
    
rows = PLOT_1_DB.query(x_max_query)
for row in rows:
    x_plot_parameters['center_max'] = row['value']

rows = PLOT_1_DB.query(y_min_query)
for row in rows:
    y_plot_parameters['center_min'] = row['value']

rows = PLOT_1_DB.query(y_max_query)
for row in rows:
    y_plot_parameters['center_max'] = row['value']
    
x_plot_parameters['range_max'] = ceil(floor(x_plot_parameters['center_max'] - x_plot_parameters['center_min']))
y_plot_parameters['range_max'] = ceil(floor(y_plot_parameters['center_max'] - y_plot_parameters['center_min']))


FIG,AX = plt.subplots()

PLOT_XCENTER_WIDGET = widgets.IntSlider(
                      value=ceil((x_plot_parameters['center_min']+x_plot_parameters['center_max'])/2),
                      min=x_plot_parameters['center_min'],
                      max=x_plot_parameters['center_max'],
                      step=1,
                      description='Plot X Center',
                      disabled=False,
                      continuous_update=False,
                      orientation='horizontal',
                      readout=True,
                      readout_format='d'
                      )

PLOT_YCENTER_WIDGET = widgets.IntSlider(
                      value=ceil((y_plot_parameters['center_min']+y_plot_parameters['center_max'])/2),
                      min=y_plot_parameters['center_min'],
                      max=y_plot_parameters['center_max'],
                      step=1,
                      description='Plot Y Center',
                      disabled=False,
                      continuous_update=False,
                      orientation='horizontal',
                      readout=True,
                      readout_format='d'
                      )

PLOT_XWIDTH_WIDGET = widgets.IntSlider(
                      value=floor(x_plot_parameters['range_max']/2),
                      min=0,
                      max=x_plot_parameters['range_max'],
                      step=1,
                      description='Plot X Width',
                      disabled=False,
                      continuous_update=False,
                      orientation='horizontal',
                      readout=True,
                      readout_format='d'
                      )

PLOT_YWIDTH_WIDGET = widgets.IntSlider(
                      value=floor(y_plot_parameters['range_max']/2),
                      min=0,
                      max=y_plot_parameters['range_max'],
                      step=1,
                      description='Plot Y Width',
                      disabled=False,
                      continuous_update=False,
                      orientation='horizontal',
                      readout=True,
                      readout_format='d'
                      )
SKIP_DELTA = widgets.IntSlider(
                      value=1,
                      min=1,
                      max=20,
                      step=1,
                      description='Skip Delta',
                      disabled=False,
                      continuous_update=False,
                      orientation='horizontal',
                      readout=True,
                      readout_format='d'
                      )

slider_box = VBox([PLOT_XCENTER_WIDGET,PLOT_XWIDTH_WIDGET,PLOT_YCENTER_WIDGET,PLOT_YWIDTH_WIDGET,SKIP_DELTA])

# Lets get a list of all of the X values.
coord = PLOT_1_DB.query("SELECT plane_origin FROM " + SLICE_AXIS )
COORD_SLICES = sorted({float(row['plane_origin']) for row in coord })

def capture_limits(b):
    global PLOT_XCENTER_WIDGET
    global PLOT_YCENTER_WIDGET
    global PLOT_XWIDTH_WIDGET
    global PLOT_YWIDTH_WIDGET
    with OUTWIDGET:
        print("In capture_limits")
    xlimits = plt.xlim()
    ylimits = plt.ylim()
    PLOT_XCENTER_WIDGET.value = ceil(xlimits[0] + xlimits[1])/2
    PLOT_YCENTER_WIDGET.value = ceil(ylimits[0] + ylimits[1])/2
    PLOT_XWIDTH_WIDGET.value = ceil(xlimits[1] - xlimits[0])
    PLOT_YWIDTH_WIDGET.value = ceil(ylimits[1] - ylimits[0])
    with OUTWIDGET:
        print("xlimits:",xlimits)
        print("ylimits:",ylimits)

def plot_coord(coord):
    global PLOT_1_DB
    global PLOT_2_DB
    global PLOT_1_SELECT
    global PLOT_2_SELECT
    global OUTWIDGET
    OUTWIDGET.clear_output()
    with OUTWIDGET:
#    if True:
#        print(PLOT_1_SELECT.value)
#        print(PLOT_2_SELECT.value)
        clear_plot = True
        if PLOT_1_SELECT.value == "Points":
            plot_1_type = "+" 
            plot_1_segments = False
#            print("Plot1 Points")
        else:
            plot_1_type = "-"
            plot_1_segments = True
#            print("Plot1 Segments")
        if PLOT_2_SELECT.value == "Points":
            plot_2_type = "r+"
            plot_2_segments = False
#            print("Plot2 Point")
        else:
            plot_2_type = "r-"
            plot_2_segments = True
#            print("Plot2 Segments")
        if PLOT_1_SELECT.value != "None":
            plot_slice(coord,PLOT_1_DB,clear_plot,plot_1_type,plot_1_segments)
            clear_plot = False
        if PLOT_2_SELECT.value != "None" and SELECT_POINTS.value == "OFF" :
            plot_slice(coord,PLOT_2_DB,clear_plot,plot_2_type,plot_2_segments)
        if PLOT_2_SELECT.value != "None" and SELECT_POINTS.value != "OFF" :
            plot_selected_points(plot_2_type)
        plt.show()
    return


def plot_selected_points(plottype):
    global SELECT_POINT_LIST
    
    x_values = []
    y_values = []
    for point in SELECT_POINT_LIST:
        x_values.append(point[0])
        y_values.append(point[1])
    plot_xcenter = PLOT_XCENTER_WIDGET.value
    plot_ycenter = PLOT_YCENTER_WIDGET.value
    plot_xwidth  = PLOT_XWIDTH_WIDGET.value
    plot_ywidth  = PLOT_YWIDTH_WIDGET.value
    plt.ylim([plot_ycenter-plot_ywidth/2,plot_ycenter+plot_ywidth/2])
    plt.xlim([plot_xcenter-plot_xwidth/2,plot_xcenter+plot_xwidth/2])
    plt.plot(x_values,y_values,linestyle='-', marker='+',)

LAST_X_VALUES = []
LAST_Y_VALUES = []

def plot_slice(slice_value,database,clear_plot,plottype,segments):
    global SLICE_AXIS
    global OUTWIDGET
    global PLTWIDGET
    global LAST_X_VALUES
    global LAST_Y_VALUES
#   with OUTWIDGET:
    if True:
        query_string = "SELECT * from "+ SLICE_AXIS + " WHERE plane_origin = " + str(slice_value)
        slice_results = database.query(query_string)
        x_plot = []
        y_plot = []
        z_plot = []
        for row in slice_results:
            x_plot.append(row['x1'])
            x_plot.append(row['x2'])
            if segments:
                x_plot.append(None)
            y_plot.append(row['y1'])
            y_plot.append(row['y2'])
            if segments:
                y_plot.append(None)
            z_plot.append(row['z1'])
            z_plot.append(row['z2'])
            if segments:
                z_plot.append(None)
    
        if clear_plot:
            plt.clf()

        plot_xcenter = PLOT_XCENTER_WIDGET.value
        plot_ycenter = PLOT_YCENTER_WIDGET.value
        plot_xwidth  = PLOT_XWIDTH_WIDGET.value
        plot_ywidth  = PLOT_YWIDTH_WIDGET.value
        plt.ylim([plot_ycenter-plot_ywidth/2,plot_ycenter+plot_ywidth/2])
        plt.xlim([plot_xcenter-plot_xwidth/2,plot_xcenter+plot_xwidth/2])
        plt.title(str(SLICE_AXIS) + " Slice Value: " + str(slice_value) )
        
        if SLICE_AXIS == "X":
            plt.plot(y_plot,z_plot,plottype)
            LAST_X_VALUES = y_plot.copy()
            LAST_Y_VALUES = z_plot.copy()
        if SLICE_AXIS == "Y":
            plt.plot(z_plot,x_plot,plottype)
            LAST_X_VALUES = z_plot.copy()
            LAST_Y_VALUES = x_plot.copy()
        if SLICE_AXIS == "Z":
            plt.plot(x_plot,y_plot,plottype)
            LAST_X_VALUES = x_plot.copy()
            LAST_Y_VALUES = y_plot.copy()

            
        
def new_plot(b):
    global COORD_SLICES
    global COORD_SLICE_WIDGET
    global SKIP_DELTA
    # Get the current value of the COORD_SLICE_WIDGET, find the next value and 
    # Set the widget to that value

    try:
        current_slice = COORD_SLICE_WIDGET.value
    except e as Exception:
        with OUTWIDGET:
            print(e)
    if b.description == "Next":
        skip_scale = 1
    elif b.description == "Previous":
        skip_scale = -1
    else:
        skip_scale = 0
    next_slice = COORD_SLICES[COORD_SLICES.index(current_slice) + skip_scale*SKIP_DELTA.value]
    COORD_SLICE_WIDGET.value = next_slice
#    with OUTWIDGET:
#        print(b.description)
    plot_coord(next_slice)

#
# Routines to select a polyline
#
#


SELECT_POINT_LIST = []


def onclick(event):
    global SELECT_POINT_LIST
    global LAST_X_VALUES
    global LAST_Y_VALUES
    with OUTWIDGET:
        print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
              ('double' if event.dblclick else 'single', event.button,
               event.x, event.y, event.xdata, event.ydata))
        print("OK, printed")
        if SELECT_POINTS.value == "SCREEN":
            if event.button == 1:
                SELECT_POINT_LIST.append([event.xdata,event.ydata])
            if event.button == 3:
                SELECT_POINT_LIST.pop()
        if SELECT_POINTS.value == "CLOSEST":
            closest_distance_squared = 1e20
            for i in range(len(LAST_X_VALUES)):
                if LAST_X_VALUES[i] != None and LAST_Y_VALUES[i] != None:
                    distance_squared = (event.ydata - LAST_Y_VALUES[i])**2 + (event.xdata - LAST_X_VALUES[i])**2
                    if distance_squared < closest_distance_squared:
                        closest_distance_squared = distance_squared
                        xdata = LAST_X_VALUES[i]
                        ydata = LAST_Y_VALUES[i]
            if event.button == 1:
                SELECT_POINT_LIST.append([xdata,ydata])
            if event.button == 3:
                closest_distance_squared = 1e20
                for i,point in enumerate(SELECT_POINT_LIST):
                    distance_squared = (event.ydata - point[1])**2 + (event.xdata - point[0])**2
                    if distance_squared < closest_distance_squared:
                        closest_distance_squared = distance_squared
                        closest_i = i
                SELECT_POINT_LIST.pop(closest_i)
        current_slice = COORD_SLICE_WIDGET.value
        plot_coord(current_slice)
    


    
def display_select_point_list(b):
    global SELECT_POINT_LIST
    with OUTWIDGET:
        print("Current Selected Point List Is:")
        for pt in SELECT_POINT_LIST:
            print(pt)

            
def clear_select_point_list(b):
    global SELECT_POINT_LIST
    with OUTWIDGET:
        print("Current Selected Point List Is:")
        SELECT_POINT_LIST = []
        
def select_points(b):
    FIG.figure.canvas.mpl_connect('button_press_event', onclick)
    with OUTWIDGET:
        print("Selecting Points")      
        print("Points Selected")
    return

def on_select_points_change(change):
    global CID
    with OUTWIDGET:
        print(change['new'])
    if change['new'] != "OFF" :
        CID = FIG.figure.canvas.mpl_connect('button_press_event', onclick)
    if change['new'] == "OFF":        
        FIG.figure.canvas.mpl_disconnect(CID)
        
def write_point_list_as_polygon(b):
    global SELECT_POINT_LIST    
    with open("polygons.scad","a") as f:
        f.write("// Polygon Created \n")
        f.write("polygon ( points = [\n")
        for count,point in enumerate(SELECT_POINT_LIST):
            if count < len(SELECT_POINT_LIST)-1:
                f.write(f"[ {point[0]} , {point[1]} ],\n")
            else:
                f.write(f"[ {point[0]} , {point[1]} ] ]);")
        
def write_point_list_as_csv(b):
    global SELECT_POINT_LIST    
    with open("line_segments.csv","a") as f:
        f.write("\# selected line segments \n")
        for point in SELECT_POINT_LIST:
           f.write(f"{point[0]} , {point[1]} \n")

        
 
    
    
        
SELECT_POINTS = RadioButtons(options = ["OFF","SCREEN","CLOSEST"],
                             description = "Select Points",
                             value="OFF",
                             disabled = False)
SELECT_POINTS.observe(on_select_points_change,names="value")

                                  

DISPLAY_POINT_LIST_BUTTON = Button(description = "Display Current Point List")
DISPLAY_POINT_LIST_BUTTON.on_click(display_select_point_list)

CLEAR_POINT_LIST_BUTTON = Button(description = "Clear Point List")
CLEAR_POINT_LIST_BUTTON.on_click(clear_select_point_list)

WRITE_POINT_LIST_SCAD_POLYGON_BUTTON = Button(description = "Write Polygon Point List")
WRITE_POINT_LIST_SCAD_POLYGON_BUTTON.on_click(write_point_list_as_polygon)

WRITE_POINT_LIST_CSV_BUTTON = Button(description = "Write CSV Point List")
WRITE_POINT_LIST_CSV_BUTTON.on_click(write_point_list_as_csv)


#
#   
#   
    
PLOT_1_SELECT = RadioButtons(options = ["None","Points","Segments"],
                             description = "File 1 Plot:",
                             value="Segments",
                             disabled = False)

PLOT_2_SELECT = RadioButtons(options = ["None","Points","Segments"],
                             description = "File 2 Plot:",
                             value="Points",
                             disabled = False)




select_box = HBox([PLOT_1_SELECT,PLOT_2_SELECT,SELECT_POINTS])
select_points_box = HBox([DISPLAY_POINT_LIST_BUTTON,
                          CLEAR_POINT_LIST_BUTTON,
                          WRITE_POINT_LIST_SCAD_POLYGON_BUTTON,
                          WRITE_POINT_LIST_CSV_BUTTON])

NEXT_BUTTON = Button(description = "Next")
NEXT_BUTTON.on_click(new_plot)
REPLOT_BUTTON = Button(description = "Replot")
REPLOT_BUTTON.on_click(new_plot)
PREVIOUS_BUTTON = Button(description = "Previous")
PREVIOUS_BUTTON.on_click(new_plot)
# SELECT_BUTTON = Button(description = "Select Points")
# SELECT_BUTTON.on_click(select_points)
CAPTURE_LIMITS_BUTTON = Button(description = "Capture Plot Limits")
CAPTURE_LIMITS_BUTTON.on_click(capture_limits)
        
button_box = HBox([PREVIOUS_BUTTON,REPLOT_BUTTON,NEXT_BUTTON,CAPTURE_LIMITS_BUTTON])    

def on_change(change):
    if change['type'] == "value":
        with OUTWIDGET:
            print(change['new'])
            
plot_widget = interactive(plot_coord,coord=COORD_SLICES)
COORD_SLICE_WIDGET = plot_widget.children[0]
COORD_SLICE_WIDGET.observe(on_change)



PLTWIDGET = widgets.Output()
OUTWIDGET = widgets.Output()
display_box = VBox([select_box,
                    PLOT_XCENTER_WIDGET,PLOT_XWIDTH_WIDGET,
                    PLOT_YCENTER_WIDGET,PLOT_YWIDTH_WIDGET,
                    SKIP_DELTA,
                    button_box,
                    select_points_box,
                    plot_widget,OUTWIDGET])
display(display_box)

# print(x_slices)